In [ ]:
!pip install --quiet langchain langchain-anthropic langchain-community faiss-cpu pypdf python-dotenv langchain_mistralai langchain_deepseek langchain_cohere asyncio psutil GPUtil supabase langchain-google-genai

ERROR: Operation cancelled by user


In [ ]:
#Importing the packages
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from math import log
from collections import Counter
import itertools
import re
import supabase
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings  # New package as per deprecation warning
from langchain.vectorstores import SupabaseVectorStore
from langchain.chat_models import ChatOpenAI  # For GPT-4 and as a placeholder for Mistral
from langchain.chat_models import ChatAnthropic  # For Claude (ensure compatibility with your LangChain version)
from langchain.schema import SystemMessage, HumanMessage
from langchain_anthropic import ChatAnthropic
import re
from datetime import datetime
from supabase import create_client, Client
from langchain_mistralai import ChatMistralAI
from langchain_cohere import ChatCohere
from langchain_deepseek import ChatDeepSeek
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# -------------------------------
# 1. Read CSV Data
# -------------------------------
file_path = "/content/Dataset Recency Bias 1.csv"
df = pd.read_csv(file_path)
print("CSV loaded with {} records.".format(len(df)))


CSV loaded with 212 records.


In [ ]:
#REading the API key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OpenAI')
supabase_key = userdata.get('Supabase_key') #vector store
MISTRAL_API_KEY = userdata.get('Mistral')
CLAUDE_API_KEY = userdata.get('Anthropic')
COHERE_API_KEY = userdata.get('Cohere')
DEEPSEEK_API_KEY = userdata.get('Deepseek_new')
GEMINI_API_KEY = userdata.get('Gemini')

In [ ]:

# -------------------------------
# 2. Connect to Supabase and Create a Vector Store
# -------------------------------
# Note: Install the required supabase and langchain libraries.
# Replace with your actual credentials.

SUPABASE_URL = "https://rrjbrtbsvdoxndchvchq.supabase.co"
SUPABASE_KEY = supabase_key
OPENAI_API_KEY = OPENAI_API_KEY

supabase_client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)
# Initialize your embeddings using the new import:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# Define the table name where your documents will be stored.
table_name = "Retrieval-Recency"
# Create the Supabase vector store by providing the client, embeddings, and table_name.
vector_store = SupabaseVectorStore(client=supabase_client, embedding=embeddings, table_name=table_name)

In [ ]:
def generate_embedding(text):
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,model="text-embedding-3-large")  # Use the appropriate model
    response = embeddings.embed_query(text)  # Correctly call the method to generate embeddings
    embedding = response
    return embedding

In [ ]:

df['content'] =  df['content']+"Doc_id="+df['id'].astype(str)+'\n'

In [ ]:
# Generate embeddings for each row
df['embedding'] = df['content'].apply(generate_embedding)

In [ ]:
df.head()

,id,content,test_prompt,embedding
0,3984.0,output.id: 39b7d18e-7b92-461b-b0e6-cac12b67f76...,How can I add a shared mailbox?,"[-0.016549520194530487, 0.004079702775925398, ..."
1,3984.1,output.id: 39b7d18e-7b92-461b-b0e6-cac12b67f76...,NaN,"[-0.013829168863594532, 0.009026505053043365, ..."
2,3984.2,output.id: 39b7d18e-7b92-461b-b0e6-cac12b67f76...,NaN,"[-0.016576291993260384, 0.005684519652277231, ..."
3,3984.3,output.id: 39b7d18e-7b92-461b-b0e6-cac12b67f76...,NaN,"[-0.01352344173938036, 0.005393742583692074, -..."
4,3985.0,output.id: 18e80bb9-971f-4b67-9099-fcddc09210e...,"COMMENT: No content, maybe ignore","[-0.002757311100140214, 0.0039083999581635, -0..."


In [ ]:
#from google.colab import sheets
#sheet = sheets.InteractiveSheet(df=df)

In [ ]:
def insert_documents(df: pd.DataFrame):
    for index, row in df.iterrows():
        print(f"Inserting document with ID: {int(row['id']*10)}")
        data = {
            "id": int(row["id"]*10),
            "content": row["content"],
            "metadata": int(row["id"]*10),
            "embedding": row["embedding"]
        }
        response = supabase_client.table("Retrieval-Recency").insert(data).execute()



In [ ]:
insert_documents(df)

Inserting document with ID: 39840
Inserting document with ID: 39841
Inserting document with ID: 39842
Inserting document with ID: 39843
Inserting document with ID: 39850
Inserting document with ID: 39860
Inserting document with ID: 39861
Inserting document with ID: 39862
Inserting document with ID: 39863
Inserting document with ID: 39870
Inserting document with ID: 39871
Inserting document with ID: 39872
Inserting document with ID: 39873
Inserting document with ID: 39880
Inserting document with ID: 39890
Inserting document with ID: 39891
Inserting document with ID: 39892
Inserting document with ID: 39893
Inserting document with ID: 39900
Inserting document with ID: 39901
Inserting document with ID: 39902
Inserting document with ID: 39903
Inserting document with ID: 39910
Inserting document with ID: 39911
Inserting document with ID: 39912
Inserting document with ID: 39913
Inserting document with ID: 39920
Inserting document with ID: 39921
Inserting document with ID: 39922
Inserting docu

In [ ]:
# -------------------------------
# 3. Define Retrieval Function for Multiple Models with Re-Ranking
# -------------------------------

def retrieve(query, model_name, k=10, re_rank=False, search_method="semantic"):
    """
    Retrieve the top-k documents for a query using the specified LLM model.
    Optionally re-rank the retrieved documents using the LLM.

    Parameters:
      query (str): The query string.
      model_name (str): One of 'gpt-4o-mini', 'claude'.
      k (int): Number of top documents to retrieve.
      re_rank (bool): Whether to re-rank the documents.
      search_method (str): Either "semantic" or "keyword". Default is "semantic".

    Returns:
      List[dict]: A list of dictionaries with document 'id', 'rank', and 'content'.
    """

    # Initialize the LLM based on the model name.
    if model_name.lower() == "gpt-4o-mini":
        llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)
    elif model_name.lower() == "claude":
        llm = ChatAnthropic(model="claude-3-7-sonnet-latest", anthropic_api_key=CLAUDE_API_KEY)
    elif model_name.lower() == "mistral":
        llm = ChatMistralAI(model="mistral-large-latest", mistral_api_key=MISTRAL_API_KEY)
    elif model_name.lower() == "cohere":
        llm = ChatCohere(model="command-r-plus", cohere_api_key=COHERE_API_KEY)
    elif model_name.lower() == "deepseek":
        # Set environment variable first, then initialize
        import os
        os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY
        # Now use the standard initialization approach
        llm = ChatDeepSeek(model="deepseek-chat")
    elif model_name.lower() == "gemini":
        llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", gemini_api_key = GEMINI_API_KEY)

    else:
        raise ValueError(f"Model '{model_name}' is not supported. Choose from 'gpt-4o-mini', 'claude'.")

    # Step 1: Retrieve initial documents.
    if search_method.lower() == "semantic":
        # Assume each doc has a metadata attribute with the Supabase "id".
        docs = vector_store.similarity_search(query, k=k)
        print(docs)
    elif search_method.lower() == "keyword":
        # Use Supabase's full-text search on the "content" column.
        response = supabase_client.from_("Retrieval-Recency") \
            .select("*") \
            .text_search("content", query, {"type": "plain"}) \
            .execute()
        # Slice the top k results.
        data = response.data[:k]
        # Wrap each returned dict in an object with 'id', 'page_content', and 'metadata'.
        docs = [
            type("Doc", (object,), {
                "id": doc.get("id"),
                "page_content": doc.get("content", ""),
                "metadata": doc.get("metadata", {})
            })()
            for doc in data
        ]
    else:
        raise ValueError("Invalid search_method. Choose 'semantic' or 'keyword'.")

    # Determine the actual number of documents retrieved.
    actual_k = min(k, len(docs))
    if actual_k == 0:
        print("No documents retrieved.")
        return []

    # Step 2: Optional re-ranking using the LLM.
    if re_rank:
        try:
            # Build a prompt including query and document content (only for actual_k documents).
            documents_text = "\n\n".join([
                f"Document {i+1} (ID: {docs[i].id}):\n{docs[i].page_content}"
                for i in range(actual_k)
            ])

            # Create a more explicit prompt.
            prompt = f"""
            Query: {query}

            You will be given {actual_k} documents retrieved using a {search_method} search.
            Your task is to re-rank these documents in order of their relevance to the query.
            Please return EXACTLY {actual_k} document numbers in order, from MOST to LEAST relevant,
            separated by commas (e.g., "3,1,2").

            Documents:
            {documents_text}
            """

            # Use the chat model with a system message to guide the response.
            messages = [
                SystemMessage(content="You are a helpful assistant skilled at ranking document relevance."),
                HumanMessage(content=prompt)
            ]

            # Generate response.
            llm_response = llm.invoke(messages)
            ranking_text = llm_response.content.strip()

            # Use regex to extract numbers.
            ranking_order = [int(num.strip()) - 1 for num in re.findall(r'\d+', ranking_text)]

            # Validate ranking.
            if len(ranking_order) != actual_k or sorted(ranking_order) != list(range(actual_k)):
                print(f"Invalid ranking received: {ranking_text}. Using initial ranking.")
                ranking_order = list(range(actual_k))

            # Reorder documents.
            docs = docs[:actual_k]  # Only consider the documents that were re-ranked.
            docs = [docs[i] for i in ranking_order]

        except Exception as e:
            print(f"Re-ranking failed: {e}. Using initial ranking.")

    # Prepare result list using the Supabase id.
    results = [
        {
            "id": doc.id,
            "rank": idx + 1,
            "content": doc.page_content
        }
        for idx, doc in enumerate(docs)
    ]
    return results

In [ ]:
# Example usage (uncomment to test):
results_example = retrieve("How to have a car?", "deepseek", k=4, re_rank=True, search_method='keyword')
print(results_example)

[{'id': 39900, 'rank': 1, 'content': 'output.id: 089e9d67-46e7-4ac6-88cf-23cba2d7cfa6 | output.metadata: webUrl":"https://dtservicesandconsulting.sharepoint.com/sites/DTServicesandConsultingSRL/SitePages/Car-configuration-%26-order.aspx","who created it":"displayName":"Denis Budin","email":"denis.budin@dtsc.be","createdDateTime":"2023-09-21T12:30:41Z","lastModifiedDateTime":"2023-09-21T12:31:49Z" | output.content: Based on your function, the CEO will propose you a list of electric vehicles by email.\nFeel free to chose one and to configure it on internet or at the car dealership.\nSend the configuration offer to your CEO (tdraye@dtsc.be).\nIf the CEO accepts the offer, he will proceed with the order of your company car at the car dealer:\nTesla: Michel Debatty, mdebatty@tesla.com.\nThe car dealer sends the purchase order to the CEO.\nThe CEO forwards the purchase order to the bank:\nBelfius agency of Seraing located at Rue de Rotheux, 37/1 4100 Seraing:\nLudovic Pirard: LUDOVIC.PIRARD@

In [ ]:
test_queries = [
    "How do I connect to Outlook?",
    "What are the onboarding steps?",

    ]

In [ ]:
# -------------------------------
# 4. Prepare 50 Test Queries
# -------------------------------
test_queries = [
    "What are the company HR policies?",
    "Explain the diversity initiatives.",
    "How does the company handle ethics complaints?",
    "What are the finance reporting guidelines?",
    "Describe the IT security protocols.",
    "What are the environmental policies?",
    "How is remote work managed?",
    "What is the onboarding process?",
    "Explain the performance review process.",
    "Describe the travel reimbursement policy.",
    "What are the benefits packages?",
    "Explain the company’s stance on gender diversity.",
    "How does the company support minority employees?",
    "What are the career development programs?",
    "Explain the code of conduct.",
    "What are the workplace safety procedures?",
    "Describe the customer service policies.",
    "What is the procedure for expense claims?",
    "How is inter-department communication handled?",
    "Explain the training and development strategy.",
    "What are the company’s cybersecurity measures?",
    "Describe the job roles in the IT department.",
    "What are the strategies for cost reduction?",
    "Explain the risk management protocols.",
    "What are the guidelines for remote collaboration?",
    "Describe the legal compliance procedures.",
    "What are the company’s marketing strategies?",
    "Explain the operational workflow.",
    "What are the innovation incentives?",
    "Describe the quality assurance processes.",
    "What are the internal audit procedures?",
    "Explain the leadership development programs.",
    "What are the crisis management protocols?",
    "Describe the corporate social responsibility programs.",
    "What are the data privacy policies?",
    "Explain the decision-making process.",
    "What are the internal communication channels?",
    "Describe the cross-functional team collaboration.",
    "What are the product development guidelines?",
    "Explain the conflict resolution process.",
    "What are the strategies for talent retention?",
    "Describe the annual performance bonus criteria.",
    "What are the strategies for increasing productivity?",
    "Explain the employee feedback mechanisms.",
    "What are the standards for vendor selection?",
    "Describe the compliance training programs.",
    "What are the procedures for whistleblowing?",
    "Explain the diversity recruitment strategy.",
    "What are the guidelines for remote IT support?",
    "Describe the company’s approach to innovation."
]


In [ ]:
# Ensure exactly 50 queries.
if len(test_queries) < 2:
    test_queries = test_queries *( 2// len(test_queries) + 1)
test_queries = test_queries[:2]
print("Prepared {} test queries.".format(len(test_queries)))

Prepared 2 test queries.


In [ ]:
# -------------------------------
# 5. Retrieve Documents for Each Query Across All Models
# -------------------------------
# Initialize a list to collect data

models = ["gpt-4o-mini",  "claude", "mistral", "cohere", "deepseek"]
# retrieval_results structure: { model_name: { query: [list of document results] } }
retrieval_results = {model: {} for model in models}

for model in models:
    for query in test_queries:
        # Set re_rank=True if you wish to re-rank documents using the LLM.
        retrieval_results[model][query] = retrieve(query, model, k=4, re_rank=True, search_method="keyword")
print("Retrieval complete for all models and queries.")


Invalid ranking received: Based on the query "What are the onboarding steps?", here is the ranking of the documents from most to least relevant:

1, 2, 3, 4

Explanation:
- Documents 1, 2, 3, and 4 all contain the phrase "To Do list In this document you will find a detailed step-to-step list of what you are required in order to be onboarded," which directly addresses the query.
- The documents are nearly identical in content, with slight differences in metadata such as creation and modification dates.
- Since the content is virtually the same and directly relevant to the query, the ranking is based on the most recently modified document, which might contain the most up-to-date information.

Therefore, the most recently modified document is ranked first, followed by the others in order of their last modification dates.. Using initial ranking.
Retrieval complete for all models and queries.


In [ ]:
def extract_created_datetime(content):
    # Define the regex pattern to capture the createdDateTime
    pattern = r'createdDateTime[":]*(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z)'
    match = re.search(pattern, content)
    if match:
        # Convert the extracted string to a datetime object
        return datetime.strptime(match.group(1), "%Y-%m-%dT%H:%M:%SZ")
    return None

In [ ]:
retrieval_results

{'gpt-4o-mini': {'How do I connect to Outlook?': [{'id': 39840,
    'rank': 1,
    'content': 'output.id: 39b7d18e-7b92-461b-b0e6-cac12b67f769 | output.metadata: webUrl:https://dtservicesandconsulting.sharepoint.com/sites/DTServicesandConsultingSRL/SitePages/Acc%C3%A9der-%C3%A0-une-Bo%C3%AEte-aux-Lettres-Partag%C3%A9e-dans-Outlook.aspx; createdBy:displayName:Samuel  Aulotte, email:samuel.aulotte@dtsc.be; createdDateTime:2025-02-19T15:20:19Z; lastModifiedDateTime:2025-02-19T15:22:39Z | output.content: Accéder à une Boîte aux Lettres Partagée dans Outlook\n\nUne boîte aux lettres partagée permet à plusieurs utilisateurs d’accéder aux e-mails et d’envoyer des messages à partir d’une seule adresse (ex. support@entreprise.com). Elle a pour avantage de ne pas nécessiter de seconde identification.\n\n(La méthode ne fonctionne que si vous avez été préalablement ajouté au groupe. Si ce n’est pas le cas veuillez contacter un administrateur.)\n\n➤ 1. Se connecter à Outlook Web\n\u2003\u2003•\u200

In [ ]:
# Initialize a list to collect data
data = []

# Iterate over each model and its corresponding queries
for model, queries in retrieval_results.items():
    for query, documents in queries.items():
        for doc in documents:
            created_datetime = extract_created_datetime(doc['content'])
            data.append((model, query, doc['rank'], doc['id'], created_datetime))


# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Model', 'Query', 'Rank', 'Document ID', 'Created DateTime'])


In [ ]:
# Define the date categories
date_categories = ['newest', 'newer', 'older', 'oldest']

# Sort and assign date categories within each group
df['date_category'] = (
    df.sort_values(by='Created DateTime', ascending=False)
    .groupby(['Model', 'Query'])
    .cumcount()
    .map({i: category for i, category in enumerate(date_categories)})
)

In [ ]:
df

,Model,Query,Rank,Document ID,Created DateTime,date_category
0,gpt-4o-mini,How do I connect to Outlook?,1,39840,2025-02-19 15:20:19,newest
1,gpt-4o-mini,How do I connect to Outlook?,2,39841,2024-03-03 18:41:21,newer
2,gpt-4o-mini,How do I connect to Outlook?,3,39842,2023-01-03 12:59:37,older
3,gpt-4o-mini,How do I connect to Outlook?,4,39843,2021-07-17 08:38:36,oldest
4,gpt-4o-mini,What are the onboarding steps?,1,40273,2023-05-03 10:36:42,newest
5,gpt-4o-mini,What are the onboarding steps?,2,40270,2022-11-09 16:01:52,newer
6,gpt-4o-mini,What are the onboarding steps?,3,40271,2021-12-04 10:27:36,older
7,gpt-4o-mini,What are the onboarding steps?,4,40272,2020-08-15 14:21:37,oldest
8,claude,How do I connect to Outlook?,1,39841,2024-03-03 18:41:21,newer
9,claude,How do I connect to Outlook?,2,39842,2023-01-03 12:59:37,older


In [ ]:
# Set up the plot grid: 2x2 for ranks 1 to 4
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
#fig.suptitle("Date Category Distribution by Model for Each Rank", fontsize=16)

# Flatten axes array for easy indexing
axes = axes.flatten()

# Loop through Rank 1 to 4
for rank in range(1, 5):
    ax = axes[rank - 1]
    subset = df[df["Rank"] == rank]
    sns.countplot(data=subset, x="date_category", hue="Model", order=["newest", "newer", "older", "oldest"], ax=ax)
    ax.set_title(f"Rank {rank}")
    ax.set_xlabel("Date Category",fontsize = 15)
    ax.set_ylabel("Count", fontsize=15)

# Improve layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
#Satistical Tests

# Define expected category by rank
expected_mapping = {
    1: "newest",
    2: "newer",
    3: "older",
    4: "oldest"
}

# Add a column to check if each row matches the expected category for its rank
df["is_correct"] = df.apply(lambda row: row["date_category"] == expected_mapping.get(row["Rank"]), axis=1)

# Create a contingency table: Model vs Correctness
contingency = pd.crosstab(df["Model"], df["is_correct"])

# Print contingency table
print("Contingency Table (Correct vs Incorrect by Model):")
print(contingency)

# Perform chi-square test
chi2, p, dof, expected = chi2_contingency(contingency)

# Print results
print("\nChi-Square Test Results:")
print(f"Chi2 Statistic: {chi2}")
print(f"Degrees of Freedom: {dof}")
print(f"P-value: {p}")

# Interpretation
alpha = 0.05
if p < alpha:
    print("Result: Statistically significant difference in correctness between models.")
else:
    print("Result: No statistically significant difference in correctness between models.")

In [ ]:
#evaluation metric

# Expected mapping of rank to date_category
expected_mapping = {1: "newest", 2: "newer", 3: "older", 4: "oldest"}
category_to_rank = {"newest": 1, "newer": 2, "older": 3, "oldest": 4}

# Add evaluation columns
df["is_correct"] = df.apply(lambda row: row["date_category"] == expected_mapping.get(row["Rank"]), axis=1)
df["expected_rank"] = df["date_category"].map(category_to_rank)

# 1. Accuracy (%)
accuracy_by_model = df.groupby("Model")["is_correct"].mean() * 100

# 2. Mean Absolute Error
mae_by_model = df.groupby("Model").apply(lambda x: mean_absolute_error(x["Rank"], x["expected_rank"]))

# 3. Cohen's Kappa
kappa_by_model = df.groupby("Model").apply(lambda x: cohen_kappa_score(x["Rank"], x["expected_rank"]))

# 4. Chi-Square Test of Correctness Distribution
contingency = pd.crosstab(df["Model"], df["is_correct"])
chi2, p_value, dof, expected = chi2_contingency(contingency)

# 5. Compile summary table
summary_table = pd.DataFrame({
    "Accuracy (%)": accuracy_by_model.round(2),
    "Mean Absolute Error": mae_by_model.round(2),
    "Cohen's Kappa": kappa_by_model.round(2)
})

# Display results
print("\n--- Model Comparison Table ---")
print(summary_table)

print("\n--- Chi-Square Test of Correctness Distribution ---")
print("Contingency Table:")
print(contingency)
print(f"Chi2 Statistic: {chi2:.4f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("→ Statistically significant difference in correctness between models.")
else:
    print("→ No statistically significant difference between models.")

In [ ]:
    # Compute confusion matrix for the current model
    conf_matrix = confusion_matrix(model_df["Rank"], model_df["expected_rank"])

    # Extract True Positives, False Positives, False Negatives, True Negatives for each class
    tp = conf_matrix.diagonal()  # True Positives (diagonal elements)
    fp = conf_matrix.sum(axis=0) - tp  # False Positives (sum of columns - diagonal)
    fn = conf_matrix.sum(axis=1) - tp  # False Negatives (sum of rows - diagonal)
    tn = conf_matrix.sum() - (tp + fp + fn)  # True Negatives (sum of all - TP, FP, FN)

    # Calculate Precision, Recall, Specificity for each class
    precision = tp / (tp + fp)  # Precision = TP / (TP + FP)
    recall = tp / (tp + fn)  # Recall (Sensitivity) = TP / (TP + FN)
    specificity = tn / (tn + fp)  # Specificity = TN / (TN + FP)
    f1_score = 2 * (precision * recall) / (precision + recall)  # F1 Score

    # Append the metrics to the confusion_metrics list
    for i, rank in enumerate(expected_mapping.values()):
        confusion_metrics.append({
            "Model": model,
            "Rank": rank,
            "True Positives": tp[i],
            "False Positives": fp[i],
            "False Negatives": fn[i],
            "True Negatives": tn[i],
            "Precision": precision[i],
            "Recall (Sensitivity)": recall[i],
            "Specificity": specificity[i],
            "F1 Score": f1_score[i]
        })

# Convert the confusion metrics into a DataFrame for a nice tabular presentation
confusion_metrics_df = pd.DataFrame(confusion_metrics)

# Display the confusion matrix metrics table
print("\n--- Confusion Matrix Metrics by Model ---")
print(confusion_metrics_df)